<a href="https://colab.research.google.com/github/mohamed32145/MindCare/blob/main/filesArrangment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
""" we will authenticate colab to access to our data on google drive to start organize and
 prepare the images fodlers

 """
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Step 3: Authenticate and create PyDrive2 client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [ ]:
"""
first step was to orginaze the data to :

CN (Cognitively Normal)
sMCI (Stable MCI)
pMCI (Progressive MCI)
AD (Alzheimer's Disease)




then we will delete the files labeled with *_scaled_2* because those file containes images
that have been adjusted to disable scaling on axes where the phantom's measurements are considered unreliable.
although *_scaled_2* are more accurate but they are less than the *_scaled* files by far and cant be mixed with the _scsale
images for the traning session
 """

#a reusable function that search and delete the scaled_2 file in the all groups (AD, CN, pMCI, sMCI)

def delete_scaled_2_folders_in_group(parent_folder_id, group_name):
    print(f"\ Processing group: {group_name}")

    # Get all subject folders inside the parent group folder (e.g., AD)
    subject_folders = drive.ListFile({
        'q': f"'{parent_folder_id}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false"
    }).GetList()

    for subject in subject_folders:
        print(f" Checking subject: {subject['title']}")

        # Get subfolders inside each subject folder
        subfolders = drive.ListFile({
            'q': f"'{subject['id']}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false"
        }).GetList()

        for folder in subfolders:
            if folder['title'].endswith('Scaled_2'):
                print(f" Deleting folder: {folder['title']} in subject {subject['title']}")
                folder.Delete()


AD_FOLDER_ID = '1OwQEPuyEC12DA9BOTHxC_9OJGoRgWYOU'
CN_FOLDER_ID = '1OwQEPuyEC12DA9BOTHxC_9OJGoRgWYOU'
PMCI_FOLDER_ID = '1OwQEPuyEC12DA9BOTHxC_9OJGoRgWYOU'
SMCI_FOLDER_ID = '1OwQEPuyEC12DA9BOTHxC_9OJGoRgWYOU'

# Call the function for each group
delete_scaled_2_folders_in_group(AD_FOLDER_ID, 'AD')
delete_scaled_2_folders_in_group(CN_FOLDER_ID, 'CN')
delete_scaled_2_folders_in_group(PMCI_FOLDER_ID, 'pMCI')
delete_scaled_2_folders_in_group(SMCI_FOLDER_ID, 'sMCI')


In [ ]:
"""
now we consider the case where a subject has two kinds of scanning sessions one
(two fodlers) one starting with MPR and the other MPR-R .MPR probably stands for "Multiplanar Reconstruction,
" a technique that generates images in multiple planes (e.g., axial, sagittal, coronal) from a single acquisition.
MPR-R may indicate a refined or revised version of the MPR protocol, potentially with improved quality or specific
modifications for certain research purposes

MPR-R is less by far than MPR, some subjects has MPR-R images more than the MPR
images, just in this case we will choose the MPR-R images

MPR-R and MPR can be mixes for the training session considering the images will
go through Preprocessing  like:

Skull Stripping

Bias Field Correction

Resampling to Common Resolution

Intensity Normalization

which we will do for all the iamges

"""

def clean_mpr_vs_mprr(drive_folder_id):
    subject_folders = drive.ListFile({'q': f"'{drive_folder_id}' in parents and trashed=false"}).GetList()

    for subject in subject_folders:
        if subject['mimeType'] != 'application/vnd.google-apps.folder':
            continue

        subject_id = subject['id']
        subject_name = subject['title']
        print(f"\n🔍 Checking subject: {subject_name}")

        subfolders = drive.ListFile({'q': f"'{subject_id}' in parents and trashed=false"}).GetList()

        mpr_folder = None
        mprr_folder = None
        mpr_count = 0
        mprr_count = 0

        for f in subfolders:
            name = f['title']
            if f['mimeType'] != 'application/vnd.google-apps.folder':
                continue

            if name.startswith('MPR__') and not name.startswith('MPR-R__'):
                contents = drive.ListFile({'q': f"'{f['id']}' in parents and trashed=false"}).GetList()
                mpr_folder = f
                mpr_count = len(contents)

            elif name.startswith('MPR-R__'):
                contents = drive.ListFile({'q': f"'{f['id']}' in parents and trashed=false"}).GetList()
                mprr_folder = f
                mprr_count = len(contents)

        if mpr_folder and mprr_folder:
            print(f" MPR: {mpr_count} files, MPR-R: {mprr_count} files")

            # Prefer MPR-R if same or more files
            if mprr_count >= mpr_count:
                print(" Deleting MPR")
                drive.CreateFile({'id': mpr_folder['id']}).Trash()
            else:
                print(" Deleting MPR-R")
                drive.CreateFile({'id': mprr_folder['id']}).Trash()
        else:
            print(" One or both MPR versions not found.")



folder_ids = {
    'AD': '1OwQEPuyEC12DA9BOTHxC_9OJGoRgWYOU',
    'PMCI': '1OwQEPuyEC12DA9BOTHxC_9OJGoRgWYOU',
    'sMCI': '1OwQEPuyEC12DA9BOTHxC_9OJGoRgWYOU',
    'CN': '1OwQEPuyEC12DA9BOTHxC_9OJGoRgWYOU''
}

for name, folder_id in folder_ids.items():
    print(f"\n Processing: {name}")
    clean_mpr_vs_mprr(folder_id)



In [ ]:
"""
as we want to train and deploy a longitudinal deep learning model its important
to orginaze the data so it would be consistent with the model structure

so we turn such .nii file path from
"ADNI\AD\002_S_0619\MPR-R__GradWarp__N3__Scaled\2007-06-22_07_06_17.0\I67871\ADNI_002_S_0619_MR_MPR-R__GradWarp__N3__Scaled_Br_20070816100717385_S33969_I67871.nii"
 into this
"ADNI_Organized\AD\002_S_0619\timepoint_1\ADNI_002_S_0619_MR_MPR-R__GradWarp__N3__Scaled_Br_20070816100717385_S33969_I67871.nii"

"""

# Load the metadata CSV
metadata_path = "https://drive.google.com/drive/folders/1yD7Yv-Gn0HluP3yAkhLuUFZn_Savy-Xh?usp=drive_link"  # or your actual path
df = pd.read_csv(metadata_path)

# Define output root directory
output_root = "C:/ADNI_Organized"

for idx, row in df.iterrows():
    group = row['group']
    subject_id = row['subject_id']
    timepoint = row['timepoint']
    original_path = row['original_path']

    # Target path
    target_dir = os.path.join(output_root, group, subject_id, f"timepoint_{timepoint}")
    os.makedirs(target_dir, exist_ok=True)

    # Copy the file
    try:
        shutil.copy(original_path, target_dir)
        print(f" Copied: {original_path} ➜ {target_dir}")
    except FileNotFoundError:
        print(f" File not found: {original_path}")
